In [1]:
import pandas as pd
import numpy as np
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [18]:
class tsp(object):
    def __init__(self, city_names=None, num_vehicles=1):
        # set city name
        self.df = pd.read_excel('cities.xlsx')
        self.all_city = self.df['name'].values
        self.num_vehicles = num_vehicles
        
        if city_names is not None:
            self.city_names = city_names
            self.df = self.df[self.df['name'].isin(city_names)]
        else:
            self.city_names = self.all_city
            
    def create_data_model(self):
        data = {}
        temp_df = pd.read_excel('distance.xlsx', index_col=0)
        temp_df = temp_df[(temp_df.index.isin(self.city_names))][self.city_names]
        data['distance_matrix'] = temp_df.values / 1000
        data['num_vehicles'] = self.num_vehicles
        data['depot'] = 0
        return data
    
    
    def get_solution(self, manager, routing, solution):
        distance_list = []
        route_list = []
        
        for vehicle_id in range(self.num_vehicles):
            # initialize distance = 0
            route_distance = 0
            route = []
            index = routing.Start(vehicle_id)
            
            while not routing.IsEnd(index):
                index_show = manager.IndexToNode(index)
                route.append(index_show)
                
                previous_index = index
                # go to next point
                index = solution.Value(routing.NextVar(index))
                route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
            route_list.append(route)
            distance_list.append(route_distance)
        return route_list, distance_list
    
    # add distance constrain
    def add_distance_dimension(self, routing, transit_callback_index):
        dimension_name = 'Distance'
        routing.AddDimension(
            transit_callback_index,
            0,
            10000,
            True,
            dimension_name)
        distance_dimension = routing.GetDimensionOrDie(dimension_name)
        distance_dimension.SetGlobalSpanCostCoefficient(100)
    
    def work(self):
        # 初始化数据
        data = self.create_data_model()

        # 创建路线管理，tsp_size（城市数量）, num_vehicles（车的数量）, depot（原点）
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                               data['num_vehicles'], data['depot'])

        # 创建 Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        # 计算两点之间的距离
        def distance_callback(from_index, to_index):
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc.
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # add distance dimension
        self.add_distance_dimension(routing, transit_callback_index)
        
        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        # 求解路径规划
        solution = routing.SolveWithParameters(search_parameters)
        # 输出结果
        route, route_distance = self.get_solution(manager, routing, solution)
        return route, route_distance

In [20]:
if __name__ == '__main__':
    model = tsp(num_vehicles=4)
    data = model.create_data_model()

    route, route_distance = model.work()
    print(route)
    print(route_distance)

[[0, 7, 18, 19, 17, 22, 6], [0, 21], [0, 5, 20, 23, 24, 26, 27, 25, 12, 32, 11, 31, 30, 13, 16], [0, 14, 28, 29, 10, 9, 8, 15, 1, 4, 2, 3]]
[6341, 7096, 6749, 6845]
